# DNN using plain Tensorflow

This notebook captures how to construct and train a DNN using low-level Tensorflow API

In [2]:
import tensorflow as tf
import numpy as np

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs =10

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
reset_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs))
y = tf.placeholder(tf.int32, shape=(None))

In [5]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [6]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [7]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [8]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [9]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [11]:
n_epochs = 40
batch_size = 50

In [12]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [13]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [16]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [18]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)
    
    save_path = saver.save(sess, "d:/ml/my_model_final.ckpt")


0 Batch accuracy: 0.94 Val accuracy: 0.916
1 Batch accuracy: 0.96 Val accuracy: 0.9352
2 Batch accuracy: 0.92 Val accuracy: 0.9456
3 Batch accuracy: 0.96 Val accuracy: 0.9532
4 Batch accuracy: 0.94 Val accuracy: 0.9554
5 Batch accuracy: 1.0 Val accuracy: 0.9592
6 Batch accuracy: 0.94 Val accuracy: 0.9618
7 Batch accuracy: 0.98 Val accuracy: 0.9634
8 Batch accuracy: 0.94 Val accuracy: 0.9646
9 Batch accuracy: 0.92 Val accuracy: 0.9668
10 Batch accuracy: 0.98 Val accuracy: 0.968
11 Batch accuracy: 0.98 Val accuracy: 0.9666
12 Batch accuracy: 0.98 Val accuracy: 0.9696
13 Batch accuracy: 1.0 Val accuracy: 0.9708
14 Batch accuracy: 0.94 Val accuracy: 0.9718
15 Batch accuracy: 1.0 Val accuracy: 0.9728
16 Batch accuracy: 1.0 Val accuracy: 0.973
17 Batch accuracy: 1.0 Val accuracy: 0.9748
18 Batch accuracy: 0.98 Val accuracy: 0.975
19 Batch accuracy: 1.0 Val accuracy: 0.9756
20 Batch accuracy: 1.0 Val accuracy: 0.9756
21 Batch accuracy: 0.98 Val accuracy: 0.9756
22 Batch accuracy: 0.98 Val acc

In [19]:
with tf.Session() as sess:
    saver.restore(sess, "d:/ml/my_model_final.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from d:/ml/my_model_final.ckpt


In [20]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
